In [26]:
import os
import yaml
import tifffile
import numpy as np

import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from src import measures
from src.separator import Separator
from src.cleaner import DoubleStepCleaner

In [27]:
main_organ = 'brain'
fpath = "/mnt/HD-LSDF/Medaka/201912_beamtime_medaka/"
cfg_path = f"/home/ws/er5241/Repos/measuring-repo/measurement_configs/measurement/{main_organ}.yaml"
db_path = "/home/ws/er5241/Repos/measuring-repo/artifacts/debug.json"

with open(cfg_path, "r") as stream:
    configs = yaml.safe_load(stream)

cleaning_config = configs['cleaning']
measuring_config = configs['measures']
centering_config = configs.get('centering', [])

In [20]:
fls = os.listdir(fpath)
id = np.random.choice(fls)

id = 'Medaka_1129_25-1'
img_path = fpath + id + "/scaled_0.5_8bit_cropped_slices.tif"
msk_path = fpath + id + f"/{main_organ}_scaled_0.5_8bit_cropped_slices.tif"

print(id)

Medaka_1129_25-1


In [21]:
img = tifffile.imread(img_path)
msk = tifffile.imread(msk_path)

In [22]:
cleaner = DoubleStepCleaner(**cleaning_config)

msk_clean, roi = cleaner(msk)
img_clean = img[roi]

In [6]:
measure_funcs = ['volume', 'volume_filled',
                 'surface_area',
                  'color_mean', 'color_mean_dilated', 'color_mean_eroded',
                  'color_median', 'color_median_dilated', 'color_median_eroded',
                  'color_std', 'color_std_dilated', 'color_std_eroded',
                  'color_perc_99', 'color_perc_99_dilated', 'color_perc_99_eroded',
                  'color_perc_1', 'color_perc_1_dilated', 'color_perc_1_eroded',
                  'convex_volume',
                  'radius_minimal_sphere', 'radius_maximal_sphere'
                 ]

In [28]:
centers = {}
for centering in centering_config:
    centers[centering['name']] = Separator((msk_clean == centering['label_id']), centering['function'], centering['count'])

In [11]:
lbl = measuring_config[5]
print(lbl['id'], lbl['name'])
lbl_mask = (msk_clean == lbl['id'])

for measure in measure_funcs:
  mf = getattr(measures, measure)
  cc = centers[lbl['center']] if ('center' in lbl.keys()) else None
  measurement = mf(lbl_mask, img_clean, cc)
  print(measure, measurement)

6 cerebellum
volume [192318]
volume_filled [192382]
surface_area [38829]
color_mean [113.01147578489793]
color_mean_dilated [80.52987253961393]
color_mean_eroded [115.90690726223932]
color_median [113.0]
color_median_dilated [69.0]
color_median_eroded [116.0]
color_std [12.895040224084553]
color_std_dilated [38.66538794965933]
color_std_eroded [11.77636111776417]
color_perc_99 [142.0]
color_perc_99_dilated [208.0]
color_perc_99_eroded [141.0]
color_perc_1 [78.0]
color_perc_1_dilated [33.0]
color_perc_1_eroded [87.0]
convex_volume [275944.0]
radius_minimal_sphere [58.43346685863325]
radius_maximal_sphere [4.771999999999991]
